# Detecter les plaques d'immatriculation des vehicules

## Import packages

In [ ]:
import torchvision.transforms.functional as TF
from torch.utils.data import DataLoader
from customDataset import CustomDataset
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
import random
import torch

# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
torch.use_deterministic_algorithms(True) # Needed for reproducible results
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"runnning on {device}")

## Preparing the dataset

In [ ]:
image_folders = [r"..\..\data\images\cars\fr", r"..\..\data\images\cars\de", r"..\..\data\images\cars\pl"]
batch_size = 16
image_size = 416
dataset = CustomDataset(image_folders, image_size=image_size)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
len(dataloader), len(dataset)

In [ ]:
# Parcourir le DataLoader pour obtenir les 6 premiers éléments
num_images = 6
images_to_display = []
annotations_to_display = []
for images, annotations in dataloader:
    for image, annotation in zip(images, annotations):
        images_to_display.append(image)
        annotations_to_display.append(annotation)
        if len(images_to_display) >= num_images:
            break
    if len(images_to_display) >= num_images:
        break
# Affichage des images
fig, axes = plt.subplots(2, 3, figsize=(12, 8))
for i, ax in enumerate(axes.flat):
    image = TF.to_pil_image(images_to_display[i])
    ax.imshow(image, cmap='gray')
    ax.axis("off")
    ax.set_title(f"Image {i+1}")
    # Dessiner les bounding boxes
    annotation = annotations_to_display[i]
    x_min, y_min, x_max, y_max = annotation.tolist()
    rect = plt.Rectangle(
        (x_min, y_min), x_max - x_min, y_max - y_min, edgecolor="r", facecolor="none"
    )
    ax.add_patch(rect)
plt.tight_layout()
plt.show()

## Building the network

In [ ]:
import torch.nn.init as init

class YOLO(nn.Module):
    def __init__(self):
        super(YOLO, self).__init__()
        # Couches de convolution pour extraire les caractéristiques de l'image
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        # Couches entièrement connectées pour prédire les bounding boxes
        self.fc1 = nn.Linear(256 * 13 * 13, 1024)
        self.fc2 = nn.Linear(1024, 4)  # 4 pour les coordonnées

        # Initialisation des poids
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    init.constant_(m.bias, 0)

    def forward(self, x):
        # Passer l'image à travers les couches de convolution
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv5(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        # # Aplatir les caractéristiques pour les passer à travers les couches entièrement connectées
        x = x.view(-1, 256 * 13 * 13)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        # # Redimensionner les sorties pour obtenir les prédictions de bounding boxes
        x = x.view(-1, 4)
        return x

# net = YOLO()
# input_data = torch.randn(13, 1, image_size, image_size)
# output_data = net(input_data)
# print("La taille de sortie :", output_data.size())
# Instanciation du modèle
model = YOLO().to(device)
print(model)

## Training the model

In [ ]:
# Optimiseur
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
# Boucle d'entraînement
num_epochs = 30
train_losses = []
n_batch = len(dataloader)
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    print(f"Epoch {epoch+1}...")
    for batch in tqdm(dataloader, total=n_batch):
        images = batch[0].to(device)
        annotations = batch[1].to(device)
        # Réinitialiser les gradients
        optimizer.zero_grad()
        # Propagation avant
        predictions = model(images)
        # Calcul de la perte
        loss = criterion(annotations, predictions)
        # Rétropropagation
        loss.backward()
        # Mise à jour des poids
        optimizer.step()
        running_loss += loss.item()
    epoch_loss = running_loss / n_batch
    train_losses.append(epoch_loss)
    print(f"Epoch [{epoch+1}/{num_epochs}], loss: {epoch_loss:.4f}")

## Evaluating the model's Performance

In [ ]:
# Create a new figure for plotting
fig = plt.figure()

# Plot training losses as a blue line
plt.plot(train_losses, color="blue", marker="o")
plt.grid(True)

# Add legend and labels
plt.legend(["Train Loss"], loc="upper right")
plt.xlabel("Epoch")
plt.ylabel("Loss")

# Display the plot
plt.show()

In [ ]:
batch_images, batch_annotations = next(iter(dataloader))
model.eval()
with torch.no_grad():
    predictions = model(batch_images)
print("MSE :", criterion(predictions, batch_annotations).item())

In [ ]:
# Affichage des images
n=2
fig, axes = plt.subplots(n, len(batch_images)//n, figsize=(12, 8))
for i, ax in enumerate(axes.flat):
    if i == len(batch_images) :
        break
    image = TF.to_pil_image(batch_images[i])
    ax.imshow(image, cmap='gray')
    ax.axis("off")
    # Dessiner les bounding boxes
    annotation = batch_annotations[i]
    annotation_pred = predictions[i]
    x_min_pred, y_min_pred, x_max_pred, y_max_pred = annotation_pred[:4].tolist()
    x_min, y_min, x_max, y_max = annotation.tolist()
    rect = plt.Rectangle(
        (x_min, y_min), x_max - x_min, y_max - y_min, edgecolor="g", facecolor="none"
    )
    rect_pred = plt.Rectangle(
        (x_min_pred, y_min_pred), x_max_pred - x_min_pred, y_max_pred - y_min_pred, edgecolor="r", facecolor="none"
    )
    ax.add_patch(rect)
    ax.add_patch(rect_pred)
    c = criterion(annotation_pred.unsqueeze(0), annotation.unsqueeze(0))
    ax.set_title(f"MSE={c.item():.2f}")
plt.tight_layout()
plt.show()